In [1]:
%load_ext autoreload
%autoreload 2

import json
import pandas as pd
import numpy as np
from keras.models import load_model
from research_lib.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
from weight_estimation.dataset import prepare_gtsf_data, compute_akpd_score
from weight_estimation.train import train, augment, normalize, get_data_split, train_model
from typing import Dict, Tuple


/home/user/miniconda/envs/py36/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


ModuleNotFoundError: No module named 'weight_estimation.dataset'

<h1> Load Raw GTSF Data </h1>

In [ ]:
s3 = S3AccessUtils('/root/data')

In [3]:
akpd_scorer_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/keypoint-detection-scorer/akpd_scorer_model_TF.h5'
akpd_scorer_f, _, _ = s3.download_from_url(akpd_scorer_url)
df1 = prepare_gtsf_data('2019-03-01', '2019-09-20', akpd_scorer_f, 0.5, 1.0)

Raw data loaded!
Data preprocessed!
Percentage complete: 0.0%
Percentage complete: 6.39%
Percentage complete: 12.78%
Percentage complete: 19.17%
Percentage complete: 25.56%
Percentage complete: 31.94%
Percentage complete: 38.33%
Percentage complete: 44.72%
Percentage complete: 51.11%
Percentage complete: 57.5%
Percentage complete: 63.89%
Percentage complete: 70.28%
Percentage complete: 76.67%
Percentage complete: 83.06%
Percentage complete: 89.45%
Percentage complete: 95.83%


In [4]:
df2 = prepare_gtsf_data('2020-06-01', '2020-08-20', akpd_scorer_f, 0.5, 1.0)

Raw data loaded!
Data preprocessed!
Percentage complete: 0.0%


/root/alok/repos/production_algo/weight_estimation/src/weight_estimation/utils.py:104: RuntimeWarning: divide by zero encountered in true_divide
  (X_left[:, 0] - X_right[:, 0])
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


In [5]:
df = pd.concat([df1, df2])

<h1> Augment the Data </h1>

In [ ]:
from weight_estimation.utils import 

In [38]:
from collections import defaultdict
from weight_estimation.utils import get_left_right_keypoint_arrs, get_ann_from_keypoint_arrs,\
    convert_to_nn_input, CameraMetadata
from weight_estimation.utils import get_left_right_keypoint_arrs, convert_to_world_point_arr


def augment(df: pd.DataFrame, augmentation_config: Dict) -> pd.DataFrame:
    print('hello')
    
    counts, edges = np.histogram(df.weight, bins=np.arange(0, 10000, 1000))
    trial_values = (5.0 / (counts / np.max(counts))).astype(int)
    max_jitter_std = augmentation_config['max_jitter_std']
    min_depth = augmentation_config['min_depth']
    max_depth = augmentation_config['max_depth']

    augmented_data = defaultdict(list)
    for idx, row in df.iterrows():
        
        camera_metadata = row.camera_metadata
        cm = CameraMetadata(
            focal_length=camera_metadata['focalLength'],
            focal_length_pixel=camera_metadata['focalLengthPixel'],
            baseline_m=camera_metadata['baseline'],
            pixel_count_width=camera_metadata['pixelCountWidth'],
            pixel_count_height=camera_metadata['pixelCountHeight'],
            image_sensor_width=camera_metadata['imageSensorWidth'],
            image_sensor_height=camera_metadata['imageSensorHeight']
        )
        
        weight = row.weight
        trials = trial_values[min(int(weight / 1000), len(trial_values) - 1)]
        for _ in range(trials):
            
            ann = row.keypoints
            X_left, X_right = get_left_right_keypoint_arrs(ann)
            wkps = convert_to_world_point_arr(X_left, X_right, cm)
            original_depth = np.median(wkps[:, 1])
            
            depth = np.random.uniform(min_depth, max_depth)
            scaling_factor = float(original_depth) / depth
            jitter_std = np.random.uniform(0, max_jitter_std)
            

            # rescale
            X_left = X_left * scaling_factor
            X_right = X_right * scaling_factor

            # add jitter
            X_left[:, 0] += np.random.normal(0, jitter_std, X_left.shape[0])
            X_right[:, 0] += np.random.normal(0, jitter_std, X_right.shape[0])

            # reconstruct annotation
            ann = get_ann_from_keypoint_arrs(X_left, X_right)
            augmented_data['annotation'].append(ann)
            augmented_data['fish_id'].append(row.fish_id)
            augmented_data['weight'].append(row.weight)
            augmented_data['kf'].append(row.k_factor)
            augmented_data['camera_metadata'].append(row.camera_metadata)

    augmented_df = pd.DataFrame(augmented_data)
    return augmented_df

ImportError: cannot import name 'get_ann_from_keypoint_arrs'

In [13]:
augmentation_config = dict(
    trials=10,
    max_jitter_std=10,
    min_depth=0.5,
    max_depth=2.5
)

augmented_df = augment(df, augmentation_config)

hello


In [14]:
augmented_df.to_csv('/root/data/sid/augmented_df_depth_weight_balanced.csv')

In [2]:
augmented_df = pd.read_csv('/root/data/sid/augmented_df_depth_weight_balanced.csv')

<h3> Note: cell below takes about 1.5 hrs to run. To load cached version, run the cell below this one </h3>

In [3]:
# count = 0
# akpd_scores = []
# akpd_scorer_network = load_model(akpd_scorer_f)
# for idx, row in augmented_df.iterrows():
#     if count % 1000 == 0:
#         print('Percentage complete: {}%'.format(round(100 * count / augmented_df.shape[0], 2)))
#     count += 1
#     akpd_score = compute_akpd_score(akpd_scorer_network, row.annotation, row.camera_metadata)
#     akpd_scores.append(akpd_score)


# augmented_df['akpd_score'] = akpd_scores

<h3> If you ran the cell above, do not run the one here </h3>

In [7]:
from tqdm import tqdm

augmented_df = pd.read_csv('/root/data/alok/biomass_estimation/playground/gtsf_augmented_dataset.csv')

new_anns, new_cms = [], []
for idx, row in tqdm(augmented_df.iterrows()):
    cm = row.camera_metadata
    new_cm = json.loads(cm.replace("'", '"'))
    new_cms.append(new_cm)
    
    ann = row.annotation
    new_ann = json.loads(ann.replace("'", '"'))
    new_anns.append(new_ann)
    
augmented_df['annotation'] = new_anns
augmented_df['camera_metadata'] = new_cms


180746it [00:39, 4539.13it/s]


<h1> Train model </h1>

In [32]:
from collections import defaultdict
import json
import os
import random
from typing import Dict, List, Tuple
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
from scipy.interpolate import interpn
from weight_estimation.utils import get_left_right_keypoint_arrs,\
    convert_to_nn_input, CameraMetadata
from keras.layers import Input, Dense, Flatten
from keras.models import Model
import keras
from research_lib.utils.data_access_utils import S3AccessUtils
from tqdm import tqdm



def normalize(anns: List, camera_metadatas: List) -> np.ndarray:
    norm_anns = []
    for ann, camera_metadata in zip(anns, camera_metadatas):

        cm = CameraMetadata(
            focal_length=camera_metadata['focalLength'],
            focal_length_pixel=camera_metadata['focalLengthPixel'],
            baseline_m=camera_metadata['baseline'],
            pixel_count_width=camera_metadata['pixelCountWidth'],
            pixel_count_height=camera_metadata['pixelCountHeight'],
            image_sensor_width=camera_metadata['imageSensorWidth'],
            image_sensor_height=camera_metadata['imageSensorHeight']
        )

        norm_ann = convert_to_nn_input(ann, cm)
        norm_anns.append(norm_ann)
    return np.array(norm_anns)


def get_data_split(X: np.ndarray, y: np.ndarray, fish_ids: np.ndarray, train_pct: float,
                   val_pct: float) -> Tuple:
    # select train / test sets such that there are no overlapping fish IDs

    test_pct = 1.0 - train_pct - val_pct
    unique_fish_ids = np.array(list(set(fish_ids)))
    train_cnt, val_cnt, test_cnt = np.random.multinomial(len(unique_fish_ids),
                                                         [train_pct, val_pct, test_pct])

    assignments = np.array([0] * train_cnt + [1] * val_cnt + [2] * test_cnt)
    np.random.shuffle(assignments)
    train_fish_ids = unique_fish_ids[np.where(assignments == 0)]
    val_fish_ids = unique_fish_ids[np.where(assignments == 1)]
    test_fish_ids = unique_fish_ids[np.where(assignments == 2)]

    train_mask = np.isin(fish_ids, train_fish_ids)
    val_mask = np.isin(fish_ids, val_fish_ids)
    test_mask = np.isin(fish_ids, test_fish_ids)

    X_train, y_train = X[train_mask], y[train_mask]
    X_val, y_val = X[val_mask], y[val_mask]
    X_test, y_test = X[test_mask], y[test_mask]

    return X_train, y_train, X_val, y_val, X_test, y_test, train_mask, val_mask, test_mask


def train_model(X_train, y_train, X_val, y_val, train_config):
    inputs = Input(shape=(24,))
    x = Dense(256, activation='relu')(inputs)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    pred = Dense(1)(x)
    model = Model(inputs, pred)

    epochs = train_config['epochs']
    batch_size = train_config['batch_size']
    lr = train_config['learning_rate']
    patience = train_config['patience']

    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                                               min_delta=0,
                                               patience=patience,
                                               verbose=0,
                                               mode='auto')]

    optimizer = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer,
                  loss='mean_squared_error',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callbacks,
              batch_size=batch_size, epochs=epochs)

    return model


def density_scatter(x, y, bins=20, **kwargs):
    fig, ax = plt.subplots(figsize=(20, 10))
    data, x_e, y_e = np.histogram2d(x, y, bins=bins, density=True)
    z = interpn((0.5*(x_e[1:] + x_e[:-1]), 0.5*(y_e[1:]+y_e[:-1])), data, np.vstack([x, y]).T,
                method="splinef2d", bounds_error=False)

    z[np.where(np.isnan(z))] = 0.0

    # Sort the points by density, so that the densest points are plotted last
    idx = z.argsort()
    x, y, z = x[idx], y[idx], z[idx]

    ax.scatter(x, y, c=z, **kwargs)

    norm = Normalize(vmin=np.min(z), vmax=np.max(z))
    cbar = fig.colorbar(cm.ScalarMappable(norm=norm), ax=ax)
    cbar.ax.set_ylabel('Density')

    ax.set_xlabel('Prediction')
    ax.set_ylabel('Ground Truth')
    ax.grid()

    return ax


def generate_accuracy_details(model, X_train, y_train, X_test, y_test, x_train_pred, y_train_pred):

    train_stats = {
        'mean_absolute_error_pct': 100 * np.mean(np.abs((y_train_pred - y_train) / y_train)),
        'mean_error_pct': 100 * np.mean(y_train_pred - y_train) / np.mean(y_train)
    }
    test_stats = {
        'mean_absolute_error_pct': 100 * np.mean(np.abs((y_test_pred - y_test) / y_test)),
        'mean_error_pct': 100 * np.mean(y_test_pred - y_test) / np.mean(y_test)
    }

    return train_stats, test_stats







In [16]:

norm_anns = []
for ann, camera_metadata in tqdm(zip(anns, cms)):

    cm = CameraMetadata(
        focal_length=camera_metadata['focalLength'],
        focal_length_pixel=camera_metadata['focalLengthPixel'],
        baseline_m=camera_metadata['baseline'],
        pixel_count_width=camera_metadata['pixelCountWidth'],
        pixel_count_height=camera_metadata['pixelCountHeight'],
        image_sensor_width=camera_metadata['imageSensorWidth'],
        image_sensor_height=camera_metadata['imageSensorHeight']
    )

    norm_ann = convert_to_nn_input(ann, cm)
    norm_anns.append(norm_ann)


180746it [00:30, 5986.14it/s]


ValueError: only one element tensors can be converted to Python scalars

In [41]:
# X = np.array()
XX = np.array([x.numpy()[0] for x in norm_anns])
XX.shape

(180746, 8, 3)

In [42]:
train_mask.shape

(180746,)

In [43]:
print('Prepping...')
# random.seed(0)
# np.random.seed(0)
# anns = augmented_df.annotation.values.tolist()
# cms = augmented_df.camera_metadata.values.tolist()
print('normalizing annotations to nn input...')
# X = normalize(anns, cms)

train_config = dict(
    train_pct=0.8,
    val_pct=0.1,
    epochs=500,
    batch_size=64,
    learning_rate=2e-5,
    patience=30
)

y = 1e-4 * augmented_df.weight.values
fish_ids = augmented_df.fish_id.values
print('Splitting...')
X_train, y_train, X_val, y_val, X_test, y_test, train_mask, val_mask, test_mask = get_data_split(XX, y, fish_ids,
                                                                train_config['train_pct'],
                                                                train_config['val_pct'])

augmented_df['train_mask'] = train_mask
augmented_df['val_mask'] = val_mask
augmented_df['test_mask'] = test_mask

Prepping...
normalizing annotations to nn input...
Splitting...


In [38]:
!rm traintest_weight_dataset.csv

In [39]:
# augmented_df.to_csv('/root/data/sid/traintest_weight_dataset.csv')

In [46]:
X_train.shape

(153487, 8, 3)

In [44]:
print('Training....')
model = train_model(X_train, y_train, X_val, y_val, train_config)

Training....
Epoch 1/500


ValueError: in user code:

    /home/user/miniconda/envs/py36/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /home/user/miniconda/envs/py36/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/user/miniconda/envs/py36/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/user/miniconda/envs/py36/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/user/miniconda/envs/py36/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/user/miniconda/env

In [35]:
fish_ids.shape

(180746,)

In [22]:
import pickle

with open('/root/data/sid/gtsf_model.pt', 'wb') as f:
    pickle.dump(model, f)

TypeError: can't pickle _thread.RLock objects

In [32]:
model

[autoreload of urllib3.connectionpool failed: Traceback (most recent call last):
  File "/home/user/miniconda/envs/py36/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/user/miniconda/envs/py36/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/user/miniconda/envs/py36/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/home/user/miniconda/envs/py36/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/user/miniconda/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py", line 11, in <module>
    from .connection

[autoreload of botocore.hooks failed: Traceback (most recent call last):
  File "/home/user/miniconda/envs/py36/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/user/miniconda/envs/py36/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/home/user/miniconda/envs/py36/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/user/miniconda/envs/py36/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "/home/user/miniconda/envs/py36/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 280, in update_instances
    ref.__class__ = new
TypeError: __class__ assignment: 'NodeList' object layout differs from 'NodeList'
]
[autoreload of botocore.docs.service failed: Traceb

In [33]:
X_train

array([[[-5.91591514e-02,  1.15923751e-02,  3.75006979e-02, ...,
          1.22998057e-01,  9.40281811e-20,  3.61096629e-02]],

       [[-5.66725699e-02,  1.02486739e-02,  2.73871426e-02, ...,
          1.04916664e-01,  2.65612392e-19,  2.62406236e-02]],

       [[-2.31413973e-01,  4.19746665e-02,  1.20768736e-01, ...,
          4.40347414e-01, -6.14641661e-19,  1.25326297e-01]],

       ...,

       [[-4.05079472e-02,  1.19912729e-02,  3.60919683e-02, ...,
          7.43890939e-02,  3.87695079e-19,  3.48870558e-02]],

       [[-8.06035287e-02,  2.65521287e-02,  8.80695432e-02, ...,
          1.76905119e-01,  2.01692632e-18,  8.80023801e-02]],

       [[-9.61349860e-02,  3.09448508e-02,  1.01042869e-01, ...,
          2.04336608e-01, -8.49254895e-19,  1.00289347e-01]]])

In [24]:
y_train_pred = model.predict(X_train).squeeze().astype(float)
y_test_pred = model.predict(X_test).squeeze().astype(float)

train_stats, test_stats = \
    generate_accuracy_details(model, X_train, y_train, X_test, y_test, y_train_pred, y_test_pred)


ValueError: operands could not be broadcast together with shapes (19171,) (139087,) 

In [ ]:
augmented_df['is_train'] = train_mask.astype(int)
augmented_df['is_val'] = val_mask.astype(int)
augmented_df['is_test'] = test_mask.astype(int)


In [ ]:
def generate_per_bucket_error(X, y):
    y_pred = model.predict(X).squeeze().astype(float)

    buckets = np.arange(0, 10000, 1000) * 1e-4
    bucket_strs = []
    mean_errs = []
    for low, high in zip(buckets, buckets[1:]):
        bucket_str = '{}-{}'.format(round(1e4 * low), round(1e4 * high))
        mask = (y >= low) & (y < high)
        mean_err = np.mean((y_pred[mask] - y[mask]) / y[mask])
        mean_errs.append(mean_err)
        bucket_strs.append(bucket_str)
    
    return pd.DataFrame({'bucket': bucket_strs, 'mean_err': mean_errs})

In [100]:
generate_per_bucket_error(X_test, y_test)

,bucket,mean_err
0,0.0-1000.0,0.021373
1,1000.0-2000.0,0.006779
2,2000.0-3000.0,-0.001876
3,3000.0-4000.0,0.035327
4,4000.0-5000.0,0.009389
5,5000.0-6000.0,0.002183
6,6000.0-7000.0,0.006544
7,7000.0-8000.0,0.026875
8,8000.0-9000.0,0.025561


In [101]:
generate_per_bucket_error(X_train, y_train)

,bucket,mean_err
0,0.0-1000.0,0.009211
1,1000.0-2000.0,0.024160
2,2000.0-3000.0,0.019169
3,3000.0-4000.0,0.019690
4,4000.0-5000.0,0.023237
5,5000.0-6000.0,0.008424
6,6000.0-7000.0,0.013830
7,7000.0-8000.0,0.007415
8,8000.0-9000.0,0.008148


In [197]:
generate_per_bucket_error(X_test, y_test)

,bucket,mean_err
0,0.0-1000.0,0.030109
1,1000.0-2000.0,-0.010025
2,2000.0-3000.0,-0.027920
3,3000.0-4000.0,0.022288
4,4000.0-5000.0,0.000550
5,5000.0-6000.0,-0.007879
6,6000.0-7000.0,-0.000133
7,7000.0-8000.0,0.010087
8,8000.0-9000.0,0.017081


In [212]:
model

<h1> Save Pytorch version of this model - the resulting output file can be then used to backtest </h1>

In [25]:
from typing import Dict, Tuple
import torch
from torch import nn
from weight_estimation.utils import CameraMetadata, convert_to_nn_input

class Network(nn.Module):
    """Network class defines neural-network architecture for both weight and k-factor estimation
    (currently both neural networks share identical architecture)."""

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(24, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        """Run inference on input keypoint tensor."""
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.output(x)
        return x
    
    def forward_intermediate(self, x):
        """Run inference on input keypoint tensor and get final hiddel layer weights."""
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        return x
        


In [26]:
pytorch_model = Network()
weights = model.get_weights()

pytorch_model.fc1.weight.data = torch.from_numpy(np.transpose(weights[0]))
pytorch_model.fc1.bias.data = torch.from_numpy(np.transpose(weights[1]))
pytorch_model.fc2.weight.data = torch.from_numpy(np.transpose(weights[2]))
pytorch_model.fc2.bias.data = torch.from_numpy(np.transpose(weights[3]))
pytorch_model.fc3.weight.data = torch.from_numpy(np.transpose(weights[4]))
pytorch_model.fc3.bias.data = torch.from_numpy(np.transpose(weights[5]))
pytorch_model.output.weight.data = torch.from_numpy(np.transpose(weights[6]))
pytorch_model.output.bias.data = torch.from_numpy(np.transpose(weights[7]))

f = '/root/data/sid/output_model.pb'
torch.save(pytorch_model.state_dict(), f)

In [31]:
model2 = Network()
model2.load_state_dict(torch.load('/root/data/sid/output_model.pb'))
model2

Network(
  (fc1): Linear(in_features=24, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (output): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
)

In [ ]:
model2

<h1> Generate errors with respect to depth </h1>

In [93]:
# get depth array and add as column to augmented data-frame

from weight_estimation.utils import get_left_right_keypoint_arrs, convert_to_world_point_arr

depths = []
for idx, row in augmented_df.iterrows():
    ann, camera_metadata = row.annotation, row.camera_metadata
    cm = CameraMetadata(
        focal_length=camera_metadata['focalLength'],
        focal_length_pixel=camera_metadata['focalLengthPixel'],
        baseline_m=camera_metadata['baseline'],
        pixel_count_width=camera_metadata['pixelCountWidth'],
        pixel_count_height=camera_metadata['pixelCountHeight'],
        image_sensor_width=camera_metadata['imageSensorWidth'],
        image_sensor_height=camera_metadata['imageSensorHeight']
    )
    
    X = convert_to_world_point_arr(*get_left_right_keypoint_arrs(ann), cm)
    median_depth = np.median(X[:, 1])
    depths.append(median_depth)
    
augmented_df['depth'] = depths

In [103]:
predictions = model(X)
augmented_df['y_pred'] = np.array(predictions).squeeze()
augmented_df['y'] = y

depths = np.arange(0.2, 2.7, 0.1)
mean_pct_errs = []
depth_buckets = []
for low_depth, high_depth in zip(depths, depths[1:]):
    depth_bucket = '{}-{}'.format(round(low_depth, 2), round(high_depth, 2))
    depth_buckets.append(depth_bucket)
    mask = (augmented_df.depth >= low_depth) & (augmented_df.depth <= high_depth)
    mean_pct_err = np.mean((augmented_df[mask].y_pred - augmented_df[mask].y) / augmented_df[mask].y)
    mean_pct_errs.append(mean_pct_err)
    

pd.DataFrame({'depth_bucket': depth_buckets, 'mean_err': mean_pct_errs})

,depth_bucket,mean_err
0,0.2-0.3,NaN
1,0.3-0.4,NaN
2,0.4-0.5,0.002595
3,0.5-0.6,0.011358
4,0.6-0.7,0.012337
5,0.7-0.8,0.008366
6,0.8-0.9,0.011369
7,0.9-1.0,0.014150
8,1.0-1.1,0.013687
9,1.1-1.2,0.011476


<h1> Perform OLS on final layer </h1>

In [134]:
from sklearn.linear_model import LinearRegression


In [135]:
X_ols = pytorch_model.forward_intermediate(torch.from_numpy(X).float()).detach().numpy()


In [136]:
lr = LinearRegression().fit(X_ols, y)

In [139]:
lr.coef_

array([ 1.06786713e-01,  3.19070697e-01, -2.83521295e-01, -5.30951321e-01,
       -1.11107302e+00, -8.43273699e-01,  1.16229057e-06,  1.06622875e-01,
        3.63074899e-01,  5.58793545e-08, -2.98023224e-08,  9.99934673e-02,
       -3.50177288e-07, -4.76837158e-07, -5.44536293e-01,  2.83486307e-01,
       -4.48852181e-02, -1.55996531e-08, -2.39884675e-01,  6.79491282e-01,
       -3.02723646e-02,  2.08616257e-07, -6.81084037e-01,  0.00000000e+00,
        3.99387747e-01,  2.37562209e-01, -4.95155334e-01, -1.19209290e-07,
       -1.63912773e-07,  8.03550333e-02,  2.08616257e-07,  4.14938122e-01,
        5.87636232e-01,  2.57033288e-01,  2.18202889e-01,  1.63912773e-07,
        1.62759155e-01,  1.92945451e-01, -1.06310844e-01, -1.78813934e-07,
       -1.79884911e-01,  3.16589445e-01,  2.08835244e-01,  9.00914490e-01,
       -2.98023224e-08,  1.83933765e-01, -6.90452993e-01,  0.00000000e+00,
       -8.14925209e-02, -4.02680814e-01, -2.69617587e-02,  6.67871892e-01,
        0.00000000e+00,  

tensor([[ 1.0679e-01,  3.1907e-01, -2.8352e-01, -5.3095e-01, -1.1111e+00,
         -8.4327e-01,  1.1623e-06,  1.0662e-01,  3.6307e-01,  5.5879e-08,
         -2.9802e-08,  9.9993e-02, -3.5018e-07, -4.7684e-07, -5.4454e-01,
          2.8349e-01, -4.4885e-02, -1.5600e-08, -2.3988e-01,  6.7949e-01,
         -3.0272e-02,  2.0862e-07, -6.8108e-01,  0.0000e+00,  3.9939e-01,
          2.3756e-01, -4.9516e-01, -1.1921e-07, -1.6391e-07,  8.0355e-02,
          2.0862e-07,  4.1494e-01,  5.8764e-01,  2.5703e-01,  2.1820e-01,
          1.6391e-07,  1.6276e-01,  1.9295e-01, -1.0631e-01, -1.7881e-07,
         -1.7988e-01,  3.1659e-01,  2.0884e-01,  9.0091e-01, -2.9802e-08,
          1.8393e-01, -6.9045e-01,  0.0000e+00, -8.1493e-02, -4.0268e-01,
         -2.6962e-02,  6.6787e-01,  0.0000e+00,  7.0650e-01,  0.0000e+00,
          2.5777e-01,  0.0000e+00,  1.6934e-01, -8.5745e-02,  8.4201e-02,
          3.6444e-02, -5.6363e-01, -2.9054e-01, -6.5735e-03]])

In [151]:
lr.intercept_

0.07646975

In [144]:
np.transpose(weights[6])

array([[ 0.2024647 ,  0.09605896,  0.3325458 ,  0.01092559, -0.2536363 ,
        -0.36302453,  0.26512384,  0.1708132 ,  0.24816908,  0.13684621,
         0.2518013 ,  0.08240502,  0.0305814 , -0.19304208, -0.4585975 ,
         0.22320633, -0.47782862,  0.29728243, -0.16720478,  0.27830702,
        -0.3700971 , -0.28571743, -0.11325362, -0.0921092 ,  0.30040807,
         0.33768034, -0.31778136,  0.18218729, -0.2121274 ,  0.2199305 ,
        -0.1901273 ,  0.3289497 , -0.23294154, -0.2563552 ,  0.0983988 ,
         0.16808727,  0.217962  ,  0.25226253, -0.28155154,  0.1247516 ,
        -0.262387  , -0.17155552,  0.30622014,  0.2477939 , -0.1691536 ,
         0.2733551 , -0.2318182 ,  0.1123949 ,  0.12816449, -0.30265012,
         0.26815662,  0.2097252 , -0.06624337,  0.1236477 ,  0.2950945 ,
         0.3244635 , -0.29667875, -0.33519307,  0.32149637,  0.14746365,
         0.30879143, -0.2418651 , -0.42719656,  0.30562183]],
      dtype=float32)

In [149]:
pytorch_model.output.weight.data = torch.from_numpy(np.array(lr.coef_).reshape(1, -1))

In [155]:
pytorch_model.output.bias.data = torch.from_numpy(np.array([lr.intercept_]))

In [162]:
preds = pytorch_model(torch.from_numpy(X).float()).detach().numpy().squeeze()

In [183]:
test_stats

{'mean_absolute_error_pct': 7.438082437849622,
 'mean_error_pct': 1.6617988080696113}

In [184]:
np.mean((pytorch_model(torch.from_numpy(X_test).float()).detach().numpy().squeeze() - y_test) / y_test)

0.009314873877741148

In [167]:
def generate_per_bucket_error_2(y_pred, y):

    buckets = np.arange(0, 10000, 1000) * 1e-4
    bucket_strs = []
    mean_errs = []
    for low, high in zip(buckets, buckets[1:]):
        bucket_str = '{}-{}'.format(round(1e4 * low), round(1e4 * high))
        mask = (y >= low) & (y < high)
        mean_err = np.mean((y_pred[mask] - y[mask]) / y[mask])
        mean_errs.append(mean_err)
        bucket_strs.append(bucket_str)
    
    return pd.DataFrame({'bucket': bucket_strs, 'mean_err': mean_errs})

In [168]:
generate_per_bucket_error_2(preds, y)

,bucket,mean_err
0,0.0-1000.0,0.035925
1,1000.0-2000.0,0.011479
2,2000.0-3000.0,0.009951
3,3000.0-4000.0,0.013634
4,4000.0-5000.0,0.013093
5,5000.0-6000.0,0.000516
6,6000.0-7000.0,0.004813
7,7000.0-8000.0,-0.004707
8,8000.0-9000.0,0.002903


In [171]:
predictions = model(X)
augmented_df['y_pred_2'] = preds
augmented_df['y'] = y

depths = np.arange(0.2, 2.7, 0.1)
mean_pct_errs = []
depth_buckets = []
for low_depth, high_depth in zip(depths, depths[1:]):
    depth_bucket = '{}-{}'.format(round(low_depth, 2), round(high_depth, 2))
    depth_buckets.append(depth_bucket)
    mask = (augmented_df.depth >= low_depth) & (augmented_df.depth <= high_depth)
    mean_pct_err = np.mean((augmented_df[mask].y_pred_2 - augmented_df[mask].y) / augmented_df[mask].y)
    mean_pct_errs.append(mean_pct_err)
    

pd.DataFrame({'depth_bucket': depth_buckets, 'mean_err': mean_pct_errs})

,depth_bucket,mean_err
0,0.2-0.3,NaN
1,0.3-0.4,NaN
2,0.4-0.5,0.001599
3,0.5-0.6,0.001967
4,0.6-0.7,0.000028
5,0.7-0.8,-0.003573
6,0.8-0.9,0.003206
7,0.9-1.0,0.005842
8,1.0-1.1,0.007641
9,1.1-1.2,0.007154


In [180]:
f = '/root/data/alok/biomass_estimation/playground/output_model_v2.pb'
torch.save(pytorch_model.state_dict(), f)